# Sensitivity Analysis

This notebook performs comprehensive sensitivity analysis on key parameters of the AI-enhanced 60/40 portfolio strategy.

## Objectives:
1. Test different lookback periods
2. Vary maximum Bitcoin allocation
3. Adjust model hyperparameters
4. Analyze robustness of results

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from itertools import product
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
from data_acquisition import DataAcquisition
from feature_engineering import FeatureEngineer
from ml_model import PortfolioMLModel
from backtester import PortfolioBacktester

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

print("Libraries imported successfully!")

## 1. Setup and Data Loading

In [ ]:
# Load configuration
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Fetch data
data_acq = DataAcquisition(config)
prices, returns, indicators = data_acq.get_full_dataset()

print(f"Data loaded: {len(prices)} periods from {prices.index[0]} to {prices.index[-1]}")

## 2. Lookback Period Sensitivity

In [ ]:
# Test different lookback periods
lookback_periods = config['sensitivity']['lookback_periods']

print(f"Testing lookback periods: {lookback_periods}")
print(f"This will take a few minutes...\n")

lookback_results = {}

for lookback in lookback_periods:
    print(f"Testing lookback period: {lookback} months")
    
    # Create features with different rolling windows based on lookback
    feature_eng = FeatureEngineer(config)
    features_raw = feature_eng.engineer_all_features(indicators)
    features = feature_eng.prepare_features_for_training(features_raw)
    
    # Train model
    ml_model = PortfolioMLModel(config)
    targets = ml_model.create_target_variables(returns, lookback=1)
    
    # Use lookback for train size
    min_train_size = lookback + 12  # Ensure enough data
    test_size = max(0.2, min_train_size / len(features))
    
    X_train, X_test, y_train, y_test = ml_model.prepare_train_test_data(
        features, targets, test_size=test_size
    )
    
    ml_model.train_all_models(X_train, y_train)
    
    # Generate allocations
    predicted_returns = ml_model.predict_returns(features)
    allocations = ml_model.calculate_optimal_allocations(predicted_returns)
    
    # Backtest
    backtester = PortfolioBacktester(config)
    results = backtester.backtest_strategy(allocations, returns, prices)
    metrics = backtester.calculate_all_metrics(results)
    
    lookback_results[f'{lookback}M'] = metrics
    
    print(f"  Sharpe Ratio: {metrics['Sharpe Ratio']:.4f}")
    print()

print("Lookback period analysis complete!")


In [ ]:
# Convert to DataFrame
lookback_df = pd.DataFrame(lookback_results).T

print("Lookback Period Sensitivity Results:")
display(lookback_df[['Sharpe Ratio', 'Sortino Ratio', 'CAGR', 'Max Drawdown', 'Volatility']])

In [ ]:
# Visualize lookback period impact
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metrics_to_plot = ['Sharpe Ratio', 'CAGR', 'Max Drawdown', 'Volatility']

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx // 2, idx % 2]
    
    values = lookback_df[metric]
    if metric == 'CAGR' or metric == 'Max Drawdown' or metric == 'Volatility':
        values = values * 100
    
    ax.plot(range(len(values)), values, marker='o', linewidth=2, markersize=8)
    ax.set_xticks(range(len(values)))
    ax.set_xticklabels(lookback_df.index)
    ax.set_xlabel('Lookback Period', fontsize=11)
    ax.set_ylabel(metric, fontsize=11)
    ax.set_title(f'{metric} by Lookback Period', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Bitcoin Allocation Sensitivity

In [ ]:
# Test different maximum Bitcoin allocations
btc_max_allocations = config['sensitivity']['btc_max_allocations']

print(f"Testing Bitcoin max allocations: {btc_max_allocations}")
print(f"This will take a few minutes...\n")

btc_results = {}

# Prepare features once
feature_eng = FeatureEngineer(config)
features_raw = feature_eng.engineer_all_features(indicators)
features = feature_eng.prepare_features_for_training(features_raw)

for max_alloc in btc_max_allocations:
    print(f"Testing max BTC allocation: {max_alloc*100:.0f}%")
    
    # Modify config temporarily
    config_modified = config.copy()
    for asset in config_modified['assets']['alternative']:
        if 'BTC' in asset['ticker']:
            asset['max_allocation'] = max_alloc
    
    # Train model
    ml_model = PortfolioMLModel(config_modified)
    targets = ml_model.create_target_variables(returns, lookback=1)
    X_train, X_test, y_train, y_test = ml_model.prepare_train_test_data(
        features, targets, test_size=0.2
    )
    
    ml_model.train_all_models(X_train, y_train)
    
    # Generate allocations
    predicted_returns = ml_model.predict_returns(features)
    allocations = ml_model.calculate_optimal_allocations(predicted_returns)
    
    # Check average BTC allocation
    if 'BTC-USD' in allocations.columns:
        avg_btc_alloc = allocations['BTC-USD'].mean()
        print(f"  Average BTC allocation: {avg_btc_alloc:.2%}")
    
    # Backtest
    backtester = PortfolioBacktester(config_modified)
    results = backtester.backtest_strategy(allocations, returns, prices)
    metrics = backtester.calculate_all_metrics(results)
    
    btc_results[f'{max_alloc*100:.0f}%'] = metrics
    
    print(f"  Sharpe Ratio: {metrics['Sharpe Ratio']:.4f}")
    print()

print("Bitcoin allocation analysis complete!")


In [ ]:
# Convert to DataFrame
btc_df = pd.DataFrame(btc_results).T

print("Bitcoin Max Allocation Sensitivity Results:")
display(btc_df[['Sharpe Ratio', 'Sortino Ratio', 'CAGR', 'Max Drawdown', 'Volatility']])

In [ ]:
# Visualize Bitcoin allocation impact
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metrics_to_plot = ['Sharpe Ratio', 'CAGR', 'Max Drawdown', 'Volatility']

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx // 2, idx % 2]
    
    values = btc_df[metric]
    if metric == 'CAGR' or metric == 'Max Drawdown' or metric == 'Volatility':
        values = values * 100
    
    ax.plot(range(len(values)), values, marker='s', linewidth=2, markersize=8, color='orange')
    ax.set_xticks(range(len(values)))
    ax.set_xticklabels(btc_df.index)
    ax.set_xlabel('Max BTC Allocation', fontsize=11)
    ax.set_ylabel(metric, fontsize=11)
    ax.set_title(f'{metric} by Max BTC Allocation', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Model Hyperparameter Sensitivity

In [ ]:
# Test different max_depth values
max_depths = config['sensitivity']['max_depths']

print(f"Testing max_depth values: {max_depths}")
print(f"This will take a few minutes...\n")

depth_results = {}

for depth in max_depths:
    print(f"Testing max_depth: {depth}")
    
    # Modify config
    config_modified = config.copy()
    config_modified['model']['parameters']['max_depth'] = depth
    
    # Train model
    ml_model = PortfolioMLModel(config_modified)
    targets = ml_model.create_target_variables(returns, lookback=1)
    X_train, X_test, y_train, y_test = ml_model.prepare_train_test_data(
        features, targets, test_size=0.2
    )
    
    ml_model.train_all_models(X_train, y_train)
    
    # Evaluate
    eval_results = ml_model.evaluate_all_models(X_test, y_test)
    avg_r2 = eval_results['r2'].mean()
    
    # Generate allocations
    predicted_returns = ml_model.predict_returns(features)
    allocations = ml_model.calculate_optimal_allocations(predicted_returns)
    
    # Backtest
    backtester = PortfolioBacktester(config)
    results = backtester.backtest_strategy(allocations, returns, prices)
    metrics = backtester.calculate_all_metrics(results)
    
    depth_results[f'depth_{depth}'] = metrics
    
    print(f"  Avg R²: {avg_r2:.4f}")
    print(f"  Sharpe Ratio: {metrics['Sharpe Ratio']:.4f}")
    print()

print("Max depth analysis complete!")

In [ ]:
# Convert to DataFrame
depth_df = pd.DataFrame(depth_results).T

print("Max Depth Sensitivity Results:")
display(depth_df[['Sharpe Ratio', 'Sortino Ratio', 'CAGR', 'Max Drawdown', 'Volatility']])

In [ ]:
# Visualize max depth impact
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metrics_to_plot = ['Sharpe Ratio', 'CAGR', 'Max Drawdown', 'Volatility']

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx // 2, idx % 2]
    
    values = depth_df[metric]
    if metric == 'CAGR' or metric == 'Max Drawdown' or metric == 'Volatility':
        values = values * 100
    
    ax.plot(range(len(values)), values, marker='^', linewidth=2, markersize=8, color='green')
    ax.set_xticks(range(len(values)))
    ax.set_xticklabels([f'{d}' for d in max_depths])
    ax.set_xlabel('Max Depth', fontsize=11)
    ax.set_ylabel(metric, fontsize=11)
    ax.set_title(f'{metric} by Max Depth', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Combined Parameter Analysis

In [ ]:
# Test different max_depth values
max_depths = config['sensitivity']['max_depths']

print(f"Testing max_depth values: {max_depths}")
print(f"This will take a few minutes...\n")

depth_results = {}

for depth in max_depths:
    print(f"Testing max_depth: {depth}")
    
    # Modify config
    config_modified = config.copy()
    config_modified['model']['parameters']['max_depth'] = depth
    
    # Train model
    ml_model = PortfolioMLModel(config_modified)
    targets = ml_model.create_target_variables(returns, lookback=1)
    X_train, X_test, y_train, y_test = ml_model.prepare_train_test_data(
        features, targets, test_size=0.2
    )
    
    ml_model.train_all_models(X_train, y_train)
    
    # Evaluate
    eval_results = ml_model.evaluate_all_models(X_test, y_test)
    avg_r2 = eval_results['r2'].mean()
    
    # Generate allocations
    predicted_returns = ml_model.predict_returns(features)
    allocations = ml_model.calculate_optimal_allocations(predicted_returns)
    
    # Backtest
    backtester = PortfolioBacktester(config)
    results = backtester.backtest_strategy(allocations, returns, prices)
    metrics = backtester.calculate_all_metrics(results)
    
    depth_results[f'depth_{depth}'] = metrics
    
    print(f"  Avg R²: {avg_r2:.4f}")
    print(f"  Sharpe Ratio: {metrics['Sharpe Ratio']:.4f}")
    print()

print("Max depth analysis complete!")


In [ ]:
# Create heatmap
fig, ax = plt.subplots(figsize=(10, 8))

sns.heatmap(combined_results, 
            annot=True, 
            fmt='.4f', 
            cmap='RdYlGn',
            xticklabels=[f'{b*100:.0f}%' for b in btc_test],
            yticklabels=[f'{l}M' for l in lookback_test],
            ax=ax,
            cbar_kws={'label': 'Sharpe Ratio'})

ax.set_xlabel('Max Bitcoin Allocation', fontsize=12)
ax.set_ylabel('Lookback Period', fontsize=12)
ax.set_title('Sharpe Ratio Heatmap: Lookback vs BTC Allocation', 
            fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Find optimal combination
max_idx = np.unravel_index(combined_results.argmax(), combined_results.shape)
optimal_lookback = lookback_test[max_idx[0]]
optimal_btc = btc_test[max_idx[1]]
optimal_sharpe = combined_results[max_idx]

print(f"\nOptimal combination:")
print(f"  Lookback: {optimal_lookback} months")
print(f"  Max BTC Allocation: {optimal_btc*100:.0f}%")
print(f"  Sharpe Ratio: {optimal_sharpe:.4f}")

## 6. Robustness Summary

In [ ]:
# Create heatmap for lookback vs BTC allocation
print("Creating combined parameter analysis...")

# Smaller grid for computational efficiency
lookback_test = [6, 12, 18]
btc_test = [0.10, 0.20, 0.30]

combined_results = np.zeros((len(lookback_test), len(btc_test)))

for i, lookback in enumerate(lookback_test):
    for j, max_btc in enumerate(btc_test):
        print(f"Testing: Lookback={lookback}M, Max BTC={max_btc*100:.0f}%")
        
        # Modify config
        config_temp = config.copy()
        for asset in config_temp['assets']['alternative']:
            if 'BTC' in asset['ticker']:
                asset['max_allocation'] = max_btc
        
        # Prepare features
        feature_eng = FeatureEngineer(config_temp)
        features_raw = feature_eng.engineer_all_features(indicators)
        features = feature_eng.prepare_features_for_training(features_raw)
        
        # Train and backtest
        ml_model = PortfolioMLModel(config_temp)
        targets = ml_model.create_target_variables(returns, lookback=1)
        
        test_size = max(0.2, (lookback + 12) / len(features))
        X_train, X_test, y_train, y_test = ml_model.prepare_train_test_data(
            features, targets, test_size=test_size
        )
        
        ml_model.train_all_models(X_train, y_train)
        predicted_returns = ml_model.predict_returns(features)
        allocations = ml_model.calculate_optimal_allocations(predicted_returns)
        
        backtester = PortfolioBacktester(config_temp)
        results = backtester.backtest_strategy(allocations, returns, prices)
        metrics = backtester.calculate_all_metrics(results)
        
        combined_results[i, j] = metrics['Sharpe Ratio']

print("\nCombined analysis complete!")


## 7. Key Findings

### Robustness Insights:

1. **Lookback Period**: 
   - The strategy shows robustness across different lookback periods
   - Shorter periods may be more reactive, longer periods more stable

2. **Bitcoin Allocation**:
   - Higher maximum BTC allocation can improve returns
   - But also increases volatility and drawdown risk
   - Sweet spot appears to be in the 20-30% range

3. **Model Complexity**:
   - Moderate tree depth (5-7) provides good balance
   - Too shallow: underfitting, too deep: overfitting

4. **Combined Effects**:
   - Parameters interact in non-linear ways
   - Optimal combination depends on risk tolerance

### Recommendations:

Based on this analysis, recommended parameter settings:
- Lookback: 12 months (balanced)
- Max BTC: 20-30% (high return potential with manageable risk)
- Max Depth: 5-7 (good generalization)